## #네이버 페이 구매내역 가져오기
**more버튼** 통해 동적으로 컨텐츠를 로드하는 페이지 형식

# 네이버 로그인

## Chromedriver 다운로드

크롬 브라우저를 제어할 수 있는 프로그램

https://chromedriver.chromium.org/downloads에서 다운로드
    
현재 사용중인 프라우저 버전에 맞는 파일 선택해야 함

## #01. 필요한 패키지 참조

> pip install --upgrade selenuim

In [38]:
import time                                      #(내장모듈) 브라우저가 페이지를 로딩하는 동안 sleep처리를 하는 모듈
from selenium import webdriver                   # 크롬 제어를 위한 모듈.  동적요소를 제어할 수 있음
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup

## #02.브라우저 제어를 위한 객체 생성

chromedriver.exe의 경로를 상대/절대 경로 형식으로 지정한다.

> 아래 블록이 실행되면 별도의 크롬브라우저가 실행된다. 해당 창을 닫으면 안된다.

In [48]:
# 객체 생성
driver = webdriver.Chrome('chromedriver.exe')
# 암묵적으로 웹 자원 로드를 위해 10초까지 기다려 준다.
driver.implicitly_wait(10)

## #03.네이버 로그인하기

1)네이버 로그인 페이지로 이동하기

In [49]:
# 네이버 로그인 페이지 접속
driver.get('https://nid.naver.com/nidlogin.login')
# 접속하는 동안 프로그램 대기(3초)
time.sleep(3)

2) 네이버 로그인 아이디, 비밀번호 입력

크롬 브라우저로 하여금 javascript 명령을 실행하도록 한다

In [50]:
myid = 'nice00001'
mypw = '7avatar3!@#'

# -> 전체문서(document)에서 id값에 의해(ById) 요소(Element)를 가져온 후 (get) 입력값(value)을 지정
script = "document.getElementById('id').value='%s'"
driver.execute_script(script % myid)

script = "document.getElementById('pw').value='%s'"
driver.execute_script(script % mypw)

3) 로그인 버튼 클릭

In [51]:
# bs4와 같은 방식으로 CSS 셀렉터가 ".bnt_global"인 요소 취득
btn = driver.find_element_by_css_selector(".btn_global")

# 취득한 버튼을 클릭시킴 --> 로그인
btn.click()

# 로그인 하는 동안 기다려 주기
time.sleep(3)

4) 로그인 완료 대기

로그인 후 페이지가 이동도니 후에 id속성이 account인 요소가 로드될 때 까지 최대 10초간 대기

In [52]:
chptcha = driver.find_element_by_css_selector("#chptcha")

if chptcha:
    user_input = input()
    script = "document.getElementById('chptcha').value = '%s'"
    driver.execute_script(script % user_input)
    
    script = "document.getElementById('pw').value='%s'"
    driver.execute_script(script % mypw)
    
    #CSS 셀렉터가 ".btn_global"인 요소 취득하여 클릭함
    driver.find_element_by_css_selector(".btn_global").click()
    
    #접속하는 동안 프로그램 대기(3초)
    time.sleep(3)

 4FDY7C6


5) 로그인 완료 대기

로그인 후 메인 페이지로 이동한 후, id속성이 account인 요소가 로드될 때까지 최대 10최간 대기

In [53]:
WebDriverWait(driver, 10).until(lambda x: x.find_element_by_id("account"))

<selenium.webdriver.remote.webelement.WebElement (session="3b095f6ea489902e8790607de3d74828", element="0bd058fb-0420-430c-9d14-0b41e8b04a16")>

--------------------------------------------------------------------------
## 04. 네이버 페이 구매내역 가져오기

1) 페이지 이동

In [54]:
#네이버 페이  메인으로 이동
driver.get("https://order.pay.naver.com/home")

#id속성이 'content'인 요소가 표시될 때까지 대기
WebDriverWait(driver, 10).until(lambda x : x.find_element_by_id("content"))

<selenium.webdriver.remote.webelement.WebElement (session="3b095f6ea489902e8790607de3d74828", element="b25c0e5b-3371-4705-92f3-6c98cb7eae09")>

2) 더보기(more) 버튼이 사라질 때까지 계속해서 클릭하기

In [55]:
# "more" 버튼이 사리질 때 까지 무한 반복
while True:
    # 버튼을 감싸고 있는 상위 요소인 "#_moreButton"를 취득 : 크롬 개발자모드에서 볼 수 있음
    more_button = driver.find_element_by_css_selector("#_moreButton")
        
    # 취득한 요소의 HTML 속성들을 딕셔너리 형태로 추출
    attrs = {}
    for item in more_button.get_property('attributes'):
        attrs[item['name']] = item['value']
        
    # more 버튼이 사라졌다면 반복 중단
    if 'style' in attrs:
        break
    
    # more 버튼을 취득하여 강제로 클릭시킴
    driver.find_element_by_css_selector(".button_viewmore").click()
    
    # 다음 컨텐츠가 로드되는 동안 약 2초간 대기
    time.sleep(1)


3) 크롬 브라우저에 나타나는 HTML 태그 가져오기

In [56]:
# 가져온 소스코드를 분석하기 위해 BeautifulSoup 객체 생성
soup = BeautifulSoup(driver.page_source, 'html.parser')

#상품정보 요소들의 List를 추출
goods = soup.select('.goods_pay_item')

goods

#브라우져 닫기
#driver.quit()

[<li class="goods_pay_item" id="_rowLi20190911092411CHK2019091122461801">
 <div class="goods_item">
 <a class="goods_thumb" href="/orderStatus/2019091135322511">
 <img alt="CT202330 [제록스/검정(대용량)/정품토너] DocuPrint M225DW M225Z M265Z P225D P225DB P265DW" height="90" src="https://order.pay.naver.com/proxy/phinf/shop1/20190825_207/heronow_1566669930395lhCqC_JPEG/4029764949842912_1992397988.jpg?type=m90_90" width="90"/> </a><!--N=a:lst.detail-->
 <div class="goods_info">
 <!-- NV_MID:82192407470 -->
 <a class="goods" href="/orderStatus/2019091135322511">
 <p class="name">
 <span class="ico_pointplus"><span class="blind">포인트플러스</span></span>
 							CT202330 [제록스/검정(대용량)/정품토너] DocuPrint M225DW M225Z M265Z P225D P225DB P265DW
 						</p>
 <ul class="info">
 <li><span class="blind">상품금액</span>66,000원</li>
 <li class="date"><span class="blind">상품구매날짜</span> 2019.09.11</li>
 </ul>
 </a><!--N=a:lst.detail-->
 <span class="state _statusName">구매확정</span>
 <p class="guide">
 <!--<p class="guide notify

## #04. 네이버 페이 주문내역 데이터 분석

1)가져온 HTML로부터 주문내역에 대한 리스트 생성

In [64]:
# 수집 결과를 저장하기 위해 빈 리스트 생성
pay_list = []

#상품정보의 수 만큼 반복
for g in goods:
    name = g.select(".name")[0].text.strip()
    name = name.replace("\n", " ")
    name = name.replace("\t", "")
    name = name.strip()
    #print(name)
    
    price = g.select(".info>li:first-child")[0].text.strip()
    price = price.replace("상품금액","")
    price = price.replace(",", "")
    price = price.replace("원","")
    price = int(price)
    #print(price)
    
    date = g.select(".info > .date")[0].text.strip()
    date = date.replace("상품구매날짜","")
    date = date.strip()
    #print(date)
    
    seller = g.select(".seller")[0].text.strip()
    #print(seller)
    
    seller_tel = g.select(".tel")[0].text.strip()
    #print(select_tel)
    
    #추출한 정보를 딕셔너리로 묶음.
    good_dict = {'구매날짜':date, '상품명':name, '금액':price, '판매처':seller, '연락처':seller_tel}
    #print(good_dict)
    
    #딕셔너리를 리스트에 병함
    pay_list.append(good_dict)
    
pay_list
    
    

[{'구매날짜': '2019.09.11',
  '상품명': '포인트플러스 CT202330 [제록스/검정(대용량)/정품토너] DocuPrint M225DW M225Z M265Z P225D P225DB P265DW',
  '금액': 66000,
  '판매처': '더셀렉션',
  '연락처': '010-4711-9994'},
 {'구매날짜': '2019.06.21',
  '상품명': '포인트플러스 에스트라 아토베리어 크림 100ml(아토베리어MD크림)',
  '금액': 54300,
  '판매처': '유산균나라',
  '연락처': '010-9698-6954'},
 {'구매날짜': '2019.06.19',
  '상품명': '포인트플러스 포켓몬스터 썬&문/포켓몬 썬문 스페셜 소장판 디럭스 시리즈/초대왕 딱지/랜덤 1개',
  '금액': 9600,
  '판매처': '한국문구 도매센터',
  '연락처': '010-7324-3113'},
 {'구매날짜': '2019.06.18',
  '상품명': '포인트플러스 [가화행거] 5단드레스룸 커튼옷걸이행거 ,바구니형행거LB4503LD',
  '금액': 92900,
  '판매처': '가화행거샵',
  '연락처': '1577-7911'},
 {'구매날짜': '2019.06.18',
  '상품명': '포인트플러스 [가화행거]2등분바구니세트/행거부품',
  '금액': 36000,
  '판매처': '가화행거샵',
  '연락처': '1577-7911'},
 {'구매날짜': '2019.06.18',
  '상품명': '포인트플러스 [가화행거]LB기둥연결 축관파이프',
  '금액': 5000,
  '판매처': '가화행거샵',
  '연락처': '1577-7911'},
 {'구매날짜': '2019.06.18',
  '상품명': '포인트플러스 [가화행거]LB기둥연결 축관파이프',
  '금액': 8000,
  '판매처': '가화행거샵',
  '연락처': '1577-7911'},
 {'구매날짜': '2019.05.26',
  '상품명': '포인트플러스 비닐봉투